## How to generate the data for DB


* **Suppliers** and **Customers** can be generated independently. 
* For *any*/*every* Supplier, some **Products** can be generated (dependence). 
* For *any*/*every* Customer, some **Orders** can be generated (dep.).
* For *any*/*every* Order, some **Order_items** and a **Shipment** can be generated (2 deps.).
    * for every **Order_item**, one existing product have to selected.

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [6]:
random_countries = [
    "Finland", "Sweden", "Norway", "Denmark", "Iceland",
    "Germany", "France", "Italy", "Spain", "Portugal",
    "Netherlands", "Belgium", "Switzerland", "Austria", "Poland",
    "Czech Republic", "Slovakia", "Hungary", "Greece", "Turkey",
    "United Kingdom", "Ireland", "Russia", "Ukraine", "Romania",
    "Bulgaria", "Croatia", "Serbia", "Slovenia", "Lithuania",
    "Latvia", "Estonia", "Belarus", "Moldova", "Albania",
    "North Macedonia", "Bosnia and Herzegovina", "Montenegro", "Kosovo",
    "Cyprus", "Malta", "Luxembourg", "Liechtenstein", "Monaco",
]

get_random_country = lambda: np.random.choice(random_countries)

In [24]:
path = "../../data/"
fname = "products.csv" 
#fname = "companies.csv"
#fname = "common-surnames-by-country.csv"
#fname = "common-forenames-by-country.csv"
#fname = "male.txt"
#fname = "female.txt"

In [25]:
data = pd.read_csv(path + fname)
data.head()

,S.No,BrandName,Product ID,Product Name,Brand Desc,Product Size,Currancy,MRP,SellPrice,Discount,Category
0,1,4711,FR001,Cologne Fragrance,ekw eau de cologne 400 ml,Small,Rs.,3900,3120,20% off,Fragrance-Women
1,2,109f,DRW1,DRW1 - Westernwear-Women,womens v- neck short dress - yellow,"Size:Medium,Small,X-Large,XX-Large",Rs.,1899,569,70% off,Westernwear-Women
2,3,109f,DRW2,DRW2 - Westernwear-Women,womens round neck solid top - black,"Size:Large,Medium,Small,X-Large",Rs.,1499,599,60% off,Westernwear-Women
3,4,109f,DRW3,DRW3 - Westernwear-Women,womens round neck stripe shift dress - red,"Size:Medium,Small",Rs.,1599,639,60% off,Westernwear-Women
4,5,109f,DRW4,DRW4 - Westernwear-Women,womens round neck solid high low top - black,"Size:Large,Medium,Small,X-Large",Rs.,1199,479,60% off,Westernwear-Women


In [27]:
for column in data.columns: print(column)

S.No
BrandName
Product ID
Product Name
Brand Desc
Product Size
Currancy
MRP
SellPrice
Discount
Category


In [30]:
as_product = lambda x: (x["Product Name"], x["Category"], round(x["SellPrice"]*0.0109,2))
as_product(data.iloc[0])

('Cologne Fragrance', 'Fragrance-Women', np.float64(34.01))

In [14]:
data.Country.unique()

array(['AM', 'AZ', 'BD', 'KH', 'CN', 'GE', 'IN', 'IL', 'JP', 'KZ', 'KR',
       'NP', 'PH', 'RU', 'LK', 'TR', 'VN', 'TW', 'AL', 'AT', 'BY', 'BE',
       'BA', 'BG', 'HR', 'CZ', 'DK', 'FO', 'EE', 'FI', 'FR', 'DE', 'GR',
       'HU', 'IS', 'IE', 'IT', 'XK', 'LV', 'LT', 'LU', 'MT', 'MD', 'ME',
       'NL', 'MK', 'NO', 'PL', 'PT', 'RO', 'RS', 'SK', 'SI', 'ES', 'SV',
       'CH', 'UA', 'GB', 'CA', 'CR', 'CU', 'DO', 'GT', 'MX', 'US', 'AU',
       'FJ', 'NZ', 'AR', 'BR', 'CL', 'CO', 'PY', 'PE', 'SR'], dtype=object)

In [23]:
locations = [item.split()[0] for item in data.Headquarters.unique()]
locations

['Mumbai',
 'Dublin',
 'Mumbai',
 'Teaneck',
 'Mumbai',
 'Bangalore',
 'Mumbai',
 'Paris',
 'Pune',
 'Noida',
 'New',
 'Navi',
 'New',
 'Bangalore',
 'New',
 'Berkshire',
 'Ahmedabad',
 'New',
 'Mumbai',
 'Washington',
 'Bangalore',
 'Mumbai',
 'Seoul',
 'New',
 'Stockholm',
 'New',
 'Mumbai',
 'Mumbai',
 'Paris',
 'Mumbai',
 'Mumbai',
 'London',
 'Navi',
 'Hyderabad',
 'Pittsford',
 'Tysons',
 'California',
 'Mumbai',
 'Gurugram',
 'Bangalore',
 'Pune',
 'Bangalore',
 'Mumbai',
 'Bengaluru',
 'Shenzhen',
 'London',
 'Noida',
 'London',
 'Espoo',
 'New',
 'Bengaluru',
 'Chennai',
 'Seoul',
 'Dongguan',
 'Ahmedabad,',
 'Kolkata',
 'Chennai',
 'Kochi',
 'Tokyo,',
 'Ann',
 'Mumbai',
 'Sttugart',
 'Hosur',
 'Bengaluru',
 'Minnesota',
 'New',
 'Bangalore',
 'Gurugram',
 'Mumbai',
 'Hyderabad',
 'New',
 'Kolkata',
 'Hyderabad',
 'California',
 'Canonsburg,Pennsylvania',
 'Troy',
 'Gurugram',
 'Florida',
 'New',
 'Noida',
 'New',
 'Mumbai',
 'Ahmedabad',
 'Texas',
 'Thrissur',
 'Gurgaon',
 'I

In [ ]:
def generate_dates(start, delta, freq='D', n=3):
    """
    Generate a list of n random dates within a specified range.
    
    Parameters:
    - start (datetime): The starting date.
    - delta (timedelta): The time delta to define the end date.
    - freq (str): Frequency string for date range generation (default is 'D' for
        daily).
    - n (int): Number of random dates to generate (default is 3).
    
    Returns:
    - tuple: A tuple containing two lists:
        - List of datetime objects.
        - List of date strings in 'YYYY-MM-DD' format.
    
    """
    end = pd.to_datetime(start+delta).date()
    seed = pd.date_range(start=start, end=end, freq=freq)
    dates = np.random.choice(seed, size=n, replace=False)
    dates = sorted(dates)
    py_dates = [pd.Timestamp(date).to_pydatetime() for date in dates]
    return py_dates, [str(date).split("T")[0] for date in dates]

In [83]:
generate_dates(datetime(2020, 1, 1), delta=timedelta(weeks=1))

([datetime.datetime(2020, 1, 2, 0, 0),
  datetime.datetime(2020, 1, 3, 0, 0),
  datetime.datetime(2020, 1, 6, 0, 0)],
 ['2020-01-02', '2020-01-03', '2020-01-06'])

In [65]:
datetime(2020, 1, 1) + timedelta(days=7)

datetime.datetime(2020, 1, 8, 0, 0)